[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ccastro1992/clustering_rfm/blob/main/notebooks/ModelamientoDBSCAN.ipynb)

# DBSCAN

### Instalacion Dependencias

In [ ]:
import shutil
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    # TRABAJAR EN EL ENTORNO DEL REPOSITORIO
    import os
    shutil.rmtree('/content/clustering_rfm')
    %cd {'/content'}

    repo_name = "clustering_rfm"

    if os.path.exists(repo_name):
        print(f"El directorio '{repo_name}' ya existe. Eliminando para clonar de nuevo...")
        shutil.rmtree(repo_name)

    #Clonar repositorio
    !git clone https://github.com/ccastro1992/{repo_name}.git

    # Entrar a la carpeta del repo
    %cd {repo_name}

In [ ]:
# ! pip install requests
# ! pip install scikit-learn-extra
# ! pip install pandas
# ! pip install matplotlib
# ! pip install seaborn

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from matplotlib import pyplot as plt
from sklearn.cluster import DBSCAN
import seaborn as sns
from src import extract
from src import transformer
from sklearn.metrics import silhouette_score
from sklearn.metrics import davies_bouldin_score

### Consulta API

In [ ]:
data = extract.get_ventas()

### 1. Carga de archivo de datos

In [ ]:
try:
    df = pd.DataFrame(data)
except Exception as e:
    print(f"Error al leer o inspeccionar el archivo: {e}")
len(df)

### 2. Conversión de Datos

In [ ]:
df = transformer.convert_data_type(df)

### 3. Filtro de Datos

In [ ]:
df_filtered = transformer.clean_data(df)

### 4. Analisis Previo

In [ ]:
# Agrupacion de ventas por fecha
grouped = df_filtered.groupby('fecha_vuelo').sum('total').sort_values('fecha_vuelo', ascending=True)
grouped = grouped.reset_index()
grouped[['fecha_vuelo', 'total']]

In [ ]:
# Tendencia de venta a lo largo de los meses
monthly_trend = grouped.set_index('fecha_vuelo')

# Re-muestrear por mes y sumar el 'total'
monthly_trend = monthly_trend['total'].resample('W').sum()

# Gráfica
plt.figure(figsize=(12, 6))
monthly_trend.plot(kind='line')
plt.title('Tendencia de Compra Semanal')
plt.xlabel('Mes')
plt.ylabel('Total de Compras')
plt.grid(True)
plt.show()

### 5. Calculo RFM Estacional

In [ ]:
# Campo de numero de semana
df_filtered['numero_semana'] = df_filtered['fecha_vuelo'].dt.isocalendar().week

In [ ]:
# Campo de fecha especial
def es_fecha_pico(numero_semana):
    if (numero_semana in [5,6,7]) or (numero_semana in [18,19]):
        return 1
    return 0

df_filtered['es_pico'] = df_filtered['numero_semana'].apply(es_fecha_pico)

In [ ]:
df_filtered['es_pico'].value_counts()

In [ ]:
# Fecha máxima de transacción + 1 dia
max_date = df_filtered['fecha_vuelo'].max() + pd.Timedelta(days=1)
print(f"Fecha Máxima: {max_date.date()}")

# Calcular Recencia, Frecuencia y Valor Monetario
rfm_df = df_filtered.groupby('cliente_id').agg(
    Recency=('fecha_vuelo', lambda x: (max_date - x.max()).days),
    Frequency=('numero_orden', 'nunique'),
    Monetary=('total', 'sum'),
    Estacional=('es_pico', 'mean')
)
rfm_df['Estacional'] = rfm_df['Estacional'].astype(int, errors='ignore')
rfm_df = rfm_df.reset_index()

### 5.1 Analisis Estadistico

In [ ]:
rfm_df.head(20)

In [ ]:
rfm_df_describe = rfm_df.describe()
rfm_df_describe

#### Recencia


In [ ]:
# Grafico estadistico Recencia
rfm_df_describe['Recency'].plot(kind='line', figsize=(8, 4), title='Recency')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(rfm_df['Recency'], bins=30, kde=True)
plt.title('Distribución de la Recencia')
plt.xlabel('Recencia (Días desde la última compra)')
plt.ylabel('Cantidad de Clientes')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(y=rfm_df['Recency'])
plt.title('Boxplot de la Recencia')
plt.ylabel('Recencia (Días desde la última compra)')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

#### Frecuencia

In [ ]:
# Grafico estadistico Frecuencia
rfm_df_describe['Frequency'].plot(kind='line', figsize=(8, 4), title='Frequency')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(y=rfm_df['Frequency'])
plt.title('Boxplot de la Frecuencia')
plt.ylabel('Frecuencia (Número de compras)')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(rfm_df['Frequency'], bins=30, kde=True)
plt.title('Distribución de la Frecuencia')
plt.xlabel('Frecuencia (Número de compras)')
plt.ylabel('Cantidad de Clientes')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

#### Monetario

In [ ]:
# Grafico estadistico Monetario
rfm_df_describe['Monetary'].plot(kind='line', figsize=(8, 4), title='Monetary')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(y=rfm_df['Monetary'])
plt.title('Boxplot del Valor Monetario')
plt.ylabel('Valor Monetario ($)')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(rfm_df['Monetary'], bins=30, kde=True)
plt.title('Distribución del Valor Monetario')
plt.xlabel('Valor Monetario ($)')
plt.ylabel('Cantidad de Clientes')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
rfm_df[['cliente_id', 'Recency', 'Frequency', 'Monetary']].sort_values('Monetary', ascending=False).head(10)

### 6. Preparación Datos

In [ ]:
### Seleccion de Caracteristicas ###
rfm_features = rfm_df[['Recency', 'Frequency', 'Monetary', 'Estacional']]

### Escalamiento y Transformacion ###
# Transformación logarítmica
rfm_log = np.log1p(rfm_features)
rfm_log.head()

In [ ]:
# Escalar
scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm_log)

In [ ]:
print(rfm_features.head())

#### 6.1 Analisis Estadistico post transformacion logaritmica

In [ ]:
# Histograma para Recency (log-transformed)
plt.subplot(1, 1, 1)
sns.histplot(rfm_log['Recency'], bins=30, kde=True)
plt.title('Distribución de Recency (Log-Transformed)')
plt.xlabel('Log(Recency)')
plt.ylabel('Cantidad de Clientes')

In [ ]:
# Histograma para Frequency (log-transformed)
plt.subplot(1, 1 ,1)
sns.histplot(rfm_log['Frequency'], bins=30, kde=True)
plt.title('Distribución de Frequency (Log-Transformed)')
plt.xlabel('Log(Frequency)')
plt.ylabel('Cantidad de Clientes')

In [ ]:
# Histograma para Monetary (log-transformed)
plt.subplot(1, 1, 1)
sns.histplot(rfm_log['Monetary'], bins=30, kde=True)
plt.title('Distribución de Monetary (Log-Transformed)')
plt.xlabel('Log(Monetary)')
plt.ylabel('Cantidad de Clientes')

plt.tight_layout()
plt.show()

### 7. Selección de k óptimo

In [ ]:
neighbors = NearestNeighbors(n_neighbors=100)
neighbors_fit = neighbors.fit(rfm_scaled)
distances, indices = neighbors_fit.kneighbors(rfm_scaled)

# Ordenar distancias y graficar
# Usamos la distancia al 8º vecino (índice 7) como un punto de partida basado en 2*dimensiones
distances = np.sort(distances[:, 7], axis=0)
plt.plot(distances)
plt.title('Gráfico de Distancia-K (K-Distance Plot) con k=100')
plt.xlabel('Puntos de datos ordenados por distancia')
plt.ylabel('Epsilon (Distancia al vecino 8)')
plt.grid(True)
plt.show()

### 8. Modelamiento DBSCAN


In [ ]:
# --- 7. Aplicar DBSCAN ---
# Aplicar DBSCAN
# eps: La distancia máxima entre dos muestras para que una se considere en la vecindad de la otra.
# min_samples: El número de muestras en una vecindad para que un punto sea considerado como un punto central.
dbscan = DBSCAN(eps=0.5, min_samples=100)
clusters = dbscan.fit_predict(rfm_scaled)

# Añadir los clusters al DataFrame original
rfm_df['Cluster'] = clusters

# Mostrar la distribución de los clusters
# El cluster -1 representa el ruido (puntos que no pertenecen a ningún cluster)
print("\n\n--- Distribución de Clusters DBSCAN ---")
print(rfm_df['Cluster'].value_counts())

In [ ]:
# Analizar los Clusters
# Calcular la media de R, F, M, E para cada cluster
cluster_analysis = rfm_df.groupby('Cluster')[['Recency', 'Frequency', 'Monetary', 'Estacional']].mean()

# Añadir el conteo de clientes en cada cluster
cluster_analysis['Count'] = rfm_df['Cluster'].value_counts()

# Reordenar columnas y ordenar por Valor Monetario para facilitar la interpretación
cluster_analysis = cluster_analysis[['Count', 'Recency', 'Frequency', 'Monetary', 'Estacional']].sort_values(
    by='Monetary', ascending=False)

# Imprimir el análisis
print("--- Análisis de Clusters (Valores Promedio por Cluster) ---")
print(cluster_analysis)

### 9. Analisis de Resultados

In [ ]:
# Silhouette Score
# Filtrar los datos para excluir el ruido (cluster -1)
# El Silhouette Score no puede manejar el clúster de ruido
labels = rfm_df['Cluster']
X_filtered = rfm_scaled[labels != -1]
labels_filtered = labels[labels != -1]

# Calcular el índice solo si hay más de 1 clúster (además del ruido)
if len(set(labels_filtered)) > 1:
    s_score = silhouette_score(X_filtered, labels_filtered)
    print(f"\n\n--- Silhouette Score ---")
    print(f"El Silhouette Score es: {s_score:.4f}")
    print("(Un valor más cercano a 1 es mejor)")
else:
    print("\n\n--- Silhouette Score ---")
    print("No se puede calcular el Silhouette Score porque no se encontró más de un clúster (excluyendo el ruido).")

In [ ]:
# Filtrar los datos para excluir el ruido (cluster -1)
# El Davies-Bouldin Score no puede manejar el clúster de ruido
labels = rfm_df['Cluster']
X_filtered = rfm_scaled[labels != -1]
labels_filtered = labels[labels != -1]

# Calcular el índice solo si hay más de 1 clúster (además del ruido)
if len(set(labels_filtered)) > 1:
    db_score = davies_bouldin_score(X_filtered, labels_filtered)
    print(f"\n\n--- Davies-Bouldin Index ---")
    print(f"El Davies-Bouldin Index es: {db_score:.4f}")
    print("(Un valor más cercano a 0 es mejor)")
else:
    print("\n\n--- Davies-Bouldin Index ---")
    print("No se puede calcular el Davies-Bouldin Index porque no se encontró más de un clúster (excluyendo el ruido).")

In [ ]:
mapeo_clientes = {
    -1: 'Clientes VIP',
    0: 'Clientes Inactivos',
    2: 'Clientes Estacionales Recurrentes',
    1: 'Clientes Estacionales (Perdidos)',
    3: 'Clientes Perdidos',
}
cluster_analysis = cluster_analysis.rename(index=mapeo_clientes)
cluster_analysis = cluster_analysis.reset_index()

### 10. Visualización

In [ ]:
plt.figure(figsize=(14, 10))

# Gráfico de Burbujas: Recencia vs. Monto (Tamaño = Cantidad de Clientes)
# Usamos escala logarítmica para el Monto porque el VIP es muy alto
plt.subplot(2, 2, 1)
sns.scatterplot(data=cluster_analysis, x='Recency', y='Monetary', size='Count', hue='Cluster', sizes=(200, 2000),
                legend=False, palette='viridis')

# Etiquetas para los puntos
for i in range(cluster_analysis.shape[0]):
    plt.text(cluster_analysis.Recency[i] + 10, cluster_analysis.Monetary[i], cluster_analysis.Cluster[i],
             horizontalalignment='left', size='medium', color='black', weight='semibold')

plt.title('Mapa de Clusters: Recencia vs. Valor Monetario\n(El tamaño de la burbuja indica la cantidad de clientes)',
          fontsize=12)
plt.xlabel('Recencia (Días desde la última compra)')
plt.ylabel('Valor Monetario Promedio ($)')
plt.yscale('log')  # Escala logarítmica para ver mejor las diferencias
plt.grid(True, which="both", ls="--", alpha=0.5)

# 2. Gráfico de Barras: Frecuencia de Compra
plt.subplot(2, 2, 2)
sns.barplot(data=cluster_analysis, x='Cluster', y='Frequency', legend=False, hue='Cluster')
plt.title('Comparación de Frecuencia de Compra', fontsize=12)
plt.ylabel('Frecuencia Promedio (# Compras)')
plt.xticks(rotation=15)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# 3. Gráfico de Barras: Cantidad de Clientes (Tamaño del Cluster)
plt.subplot(2, 2, 3)
sns.barplot(data=cluster_analysis, x='Cluster', y='Count', legend=False, hue='Cluster')
plt.title('Tamaño de cada Cluster (Cantidad de Clientes)', fontsize=12)
plt.ylabel('Número de Clientes')
plt.xticks(rotation=15)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# 4. Gráfico de Barras: Valor Monetario
plt.subplot(2, 2, 4)
sns.barplot(data=cluster_analysis, x='Cluster', y='Monetary', legend=False, hue='Cluster')
plt.title('Comparación de Gasto Promedio (Monetary)', fontsize=12)
plt.ylabel('Monto Promedio ($)')
plt.xticks(rotation=15)
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

In [ ]:
# --- Visualización Individual de Clusters ---
# Mapear los números de cluster a los nombres descriptivos en el dataframe principal
rfm_df['Cluster_Name'] = rfm_df['Cluster'].map(mapeo_clientes)

# Obtener los nombres únicos de los clusters
cluster_names = rfm_df['Cluster_Name'].unique()

for cluster_name in cluster_names:
    if pd.isna(cluster_name):
        continue

    # Filtrar los datos para el cluster actual
    cluster_data = rfm_df[rfm_df['Cluster_Name'] == cluster_name]

    plt.figure(figsize=(12, 8))

    # Grafico de dispersión para el cluster actual
    sns.scatterplot(
        data=cluster_data,
        x='Recency',
        y='Monetary',
        size='Frequency',
        hue='Frequency',
        sizes=(50, 1000),
        palette='viridis',
        legend='auto'
    )

    plt.title(f'Análisis del Cluster: "{cluster_name}" ({len(cluster_data)} clientes)', fontsize=16)
    plt.xlabel('Recencia (Días desde la última compra)', fontsize=12)
    plt.ylabel('Valor Monetario Total ($)', fontsize=12)
    plt.grid(True, which="both", ls="--", alpha=0.5)

    # Anotar algunos clientes para identificación
    top_clients = cluster_data.nlargest(5, 'Monetary')

    for index, row in top_clients.iterrows():
        plt.text(row['Recency'] + 5, row['Monetary'], str(int(row['cliente_id'])),
                 horizontalalignment='left', size='small', color='black', weight='semibold')

    plt.legend(title='Frecuencia')
    plt.show()